In [1]:
rm -rf obitools4
git clone https://github.com/metabarcoding/obitools4 

Cloning into 'obitools4'...
remote: Enumerating objects: 7574, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 7574 (delta 287), reused 384 (delta 148), pack-reused 6983 (from 2)
Receiving objects: 100% (7574/7574), 188.68 MiB | 29.67 MiB/s, done.
Resolving deltas: 100% (4721/4721), done.
Updating files: 100% (529/529), done.


In [2]:
### TEST FIXER LA VERSION 
cd obitools4
git reset --hard 996ec69
cd ..

HEAD is now at 996ec69 update the release notes for version 4.4.0


In [3]:
conda install -c conda-forge compilers=1.10 make=4.4.1 go=1.24.5 zlib=1.3.1 jq=1.8.1 --yes

Retrieving notices: done
Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - compilers=1.10
    - go=1.24.5
    - jq=1.8.1
    - make=4.4.1
    - zlib=1.3.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _go_select-2.2.0           |            nocgo           5 KB  conda-forge
    binutils-2.43              |       h4852527_5          34 KB  conda-forge
    binutils_linux-64-2.43     |       h4852527_4          35 KB  conda-forge
    c-compiler-1.10.0          |       h2b85faf_0           7 KB  conda-forge
    ca-certificates-2025.8.3   |       hbd8a1cb_0         151 KB  conda-forge

In [4]:
# CHANGER LE LIEN DU GITLAB EN GITHUB
find . -type f -exec sed -i \
  -e 's|git\.metabarcoding\.org/obitools/obitools4/obitools4|github.com/metabarcoding/obitools4|g' \
  -e 's|git\.metabarcoding\.org/org-asm/org-asm\.git|github.com/metabarcoding/org-asm.git|g' \
  -e 's|git\.metabarcoding\.org/lecasofts/go/obitools|github.com/metabarcoding/obitools4|g' \
  -e 's|https://git\.metabarcoding\.org|https://github.com/metabarcoding|g' \
  {} +


In [5]:
export CC=$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-cc
export CGO_ENABLED=1
export CGO_CFLAGS="-I$CONDA_PREFIX/include"
export CGO_LDFLAGS="-L$CONDA_PREFIX/lib"

In [6]:
cd obitools4
make
cd ..

mkdir -p build
Update version.go : pkg/obioptions/version.go to Release 4.4.0 (996ec69)
- Building obitool obiannotate...Done.
- Building obitool obiclean...Done.
- Building obitool obicleandb...Done.
- Building obitool obicomplement...Done.
- Building obitool obiconsensus...Done.
- Building obitool obiconvert...Done.
- Building obitool obicount...Done.
- Building obitool obicsv...Done.
- Building obitool obidemerge...Done.
- Building obitool obidistribute...Done.
- Building obitool obigrep...Done.
- Building obitool obijoin...Done.
- Building obitool obikmermatch...Done.
- Building obitool obikmersimcount...Done.
- Building obitool obilandmark...Done.
- Building obitool obimatrix...Done.
- Building obitool obimicrosat...Done.
- Building obitool obimultiplex...Done.
- Building obitool obipairing...Done.
- Building obitool obipcr...Done.
- Building obitool obireffamidx...Done.
- Building obitool obirefidx...Done.
- Building obitool obiscript...Done.
- Building obitool obisplit...Done.
-

In [ ]:
## FORMATAGE DES DONNES / ESPACES RETIRE
shopt -s nullglob
for f in galaxy_inputs/inputfile/*.{fastq*,fasta}; do
  [ -e "$f" ] || continue
  dir=$(dirname -- "$f")
  base=$(basename -- "$f")
  # on enlève les espaces
  newbase=${base// /_}
  newpath="$dir/$newbase"
  [ "$newpath" = "$f" ] && continue
  echo "Renomme: $f -> $newpath"
  mv -- "$f" "$newpath"
done

In [ ]:
# Recherche des chemins des inputs et formatage de l'ouput 
jq '.inputfile[0].path |= gsub(" "; "_")' galaxy_inputs/galaxy_inputs.json >  galaxy_inputs/galaxy_inputs_fixed.json
json_path="galaxy_inputs/galaxy_inputs_fixed.json"

# input_file="galaxy_inputs/wolfdiet_obigrep.fastq" test injection d'input
input_file=$(grep -o '"path": *"[^"]*"' "$json_path" | head -n 1 | sed 's/.*"path": *"\([^"]*\)".*/\1/')


echo "Input file path: $input_file"


filename=$(basename -- "$input_file")       # ex: myfile.fastq.gz
extension="${filename##*.}"                # ex: gz
filename="${filename%.*}"                  # ex: myfile.fastq ou myfile

echo "Nom de fichier (sans chemin, sans extension) : $filename"
echo "Extension détectée : $extension"


output_file="outputs/outputfile.$extension"

format=""


Input file path: galaxy_inputs/inputfile/Banque1_ali.fastqsanger
Nom de fichier (sans chemin, sans extension) : Banque1_ali
Extension détectée : fastqsanger


In [ ]:
## Chemin ngs filter
ngs_file=$(jq -r '.ngs[0].path' galaxy_inputs/galaxy_inputs.json)
ngs_file=$(printf "%s" "$ngs_file" | sed 's/[[:space:]]//g')

In [ ]:
# Si le nbre de mismatch signifié par l'utilisateur -> le met en paramètre sinon valeur par défaut (-1)
mis=$(jq -r '.AllowedXmismatches' galaxy_inputs/galaxy_inputs.json)

if [[ -n "$mis"  && "$mis" != "null" ]]; then
    mis="-e $mis"
    echo "variable mis présente: $mis"
else 
    mis=""
fi

In [ ]:
# Unidentified STRING Filename to store unassigned to any sample


# if [[ unidentified == "True" ]]

In [ ]:
./obitools4/build/obimultiplex $mis -s $ngs_file -u outputs/unidentified.fastq $input_file > $output_file 

INFO[0000] Number of workers set 120                    
ERRO[0000] Cannot open file null: cannot open path      


: 1